# 영어 Word2Vec 만들기

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

## 훈련 데이터 이해하기

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f40ec9bd9d0>)

In [ ]:
!ls -l ted_en-20160408.xml # 74MB

-rw-r--r-- 1 root root 74533638 Sep  3 04:17 ted_en-20160408.xml


In [ ]:
!more ted_en-20160408.xml 
# 러닝하고 q입력하기, xml : eXtensible Markup Language-문서 교환의 표준 --> json-현재 문서 교환의 표준
# 훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요
# XMl의 형식 : <CODE>INFO-000</CODE>
# json의 형식 : "CODE" : "INFO-000"

<?xml version="1.0" encoding="UTF-8"?>
<xml language="en"><file id="1">
  <head>
    <url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_ho
w_to_avoid_them</url>
    <pagesize>72832</pagesize>
    <dtime>Fri Apr 01 00:57:03 CEST 2016</dtime>
    <encoding>UTF-8</encoding>
    <content-type>text/html; charset=utf-8</content-type>
    <keywords>talks, business, creativity, curiosity, goal-setting, innovation, 
motivation, potential, success, work</keywords>
    <speaker>Knut Haanaes</speaker>
    <talkid>2470</talkid>
    <videourl>http://download.ted.com/talks/KnutHaanaes_2015S.mp4</videourl>
    <videopath>talks/KnutHaanaes_2015S.mp4</videopath>
    <date>2015/06/30</date>
    <title>Knut Haanaes: Two reasons companies fail -- and how to avoid them</ti
tle>
    <description>TED Talk Subtitles and Transcript: Is it possible to run a comp
any and reinvent it at the same time? For business strategist Knut Haanaes, the 
ability to innovate after becoming successful is

## 훈련 데이터 전처리하기

In [ ]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [ ]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [ ]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [ ]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print(f'총 샘플의 개수 : {len(result):,d}')

총 샘플의 개수 : 273,424


In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련시키기
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model.wv.most_similar('man')

[('woman', 0.8396265506744385),
 ('guy', 0.8157703280448914),
 ('lady', 0.7985696792602539),
 ('boy', 0.7604172229766846),
 ('girl', 0.7434580326080322),
 ('soldier', 0.727820634841919),
 ('gentleman', 0.7168433666229248),
 ('kid', 0.6915112733840942),
 ('rabbi', 0.6906076073646545),
 ('friend', 0.6810787916183472)]

In [ ]:
# man - boy + girl --> woman
model.wv.most_similar(positive=['man', 'girl'], negative=['boy'], topn=3)

[('woman', 0.846427321434021),
 ('lady', 0.7639711499214172),
 ('guy', 0.7409915924072266)]

## 모델 저장하고 로드하기

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [ ]:
!ls -l

total 97900
-rw-r--r-- 1 root root 25707730 Sep  3 04:44 eng_w2v
drwxr-xr-x 1 root root     4096 Sep  1 19:26 sample_data
-rw-r--r-- 1 root root 74533638 Sep  3 04:17 ted_en-20160408.xml


In [ ]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
loaded_model.most_similar('man')

[('woman', 0.8396265506744385),
 ('guy', 0.8157703280448914),
 ('lady', 0.7985696792602539),
 ('boy', 0.7604172229766846),
 ('girl', 0.7434580326080322),
 ('soldier', 0.727820634841919),
 ('gentleman', 0.7168433666229248),
 ('kid', 0.6915112733840942),
 ('rabbi', 0.6906076073646545),
 ('friend', 0.6810787916183472)]